# Segmenting and Clustering Neighborhoods in Toronto

**In this project, we will explore, segment, and cluster the neighborhoods in the city of Toronto. Since, the data is not readily available on the internet, we will scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format.**


Importing necessary Libraries

In [0]:
import pandas as pd
import numpy as np

### 1. Code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

We will read the table at the link - https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M  into the pandas dataframe

We will first import requests library which will help you in sending a request to the URL from where you want to fetch the HTML content.

In [35]:
import requests
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
postal_url = requests.get(url)
postal_url

<Response [200]>

Response [200] OK means that we were able to connect with that web server.


Now, we will read the HTML and store it into the dataFrame

In [36]:
postal_data = pd.read_html(postal_url.text)
len(postal_data), type(postal_data)

(3, list)

In [37]:
postal_data

[    Postal code  ...                                       Neighborhood
 0           M1A  ...                                                NaN
 1           M2A  ...                                                NaN
 2           M3A  ...                                          Parkwoods
 3           M4A  ...                                   Victoria Village
 4           M5A  ...                         Regent Park / Harbourfront
 ..          ...  ...                                                ...
 175         M5Z  ...                                                NaN
 176         M6Z  ...                                                NaN
 177         M7Z  ...                                                NaN
 178         M8Z  ...  Mimico NW / The Queensway West / South of Bloo...
 179         M9Z  ...                                                NaN
 
 [180 rows x 3 columns],
                                                   0   ...   17
 0                               

From the above output, it is clear that there are 3 tables with a type list. We only need the table 1. We will drop the other tables.

In [38]:
postal_data = postal_data[0]
postal_data

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
...,...,...,...
175,M5Z,Not assigned,NaN
176,M6Z,Not assigned,NaN
177,M7Z,Not assigned,NaN
178,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...


So now we have the table in our dataframe. Now we have to perform following cleaning of data:

1. Remove cells with a borough that is Not assigned.

In [39]:
unassigned_borough = postal_data[postal_data["Borough"] == "Not assigned"].index
unassigned_borough

Int64Index([  0,   1,   7,  10,  15,  16,  19,  24,  25,  28,  29,  33,  34,
             35,  37,  38,  42,  43,  44,  51,  52,  53,  60,  61,  62,  69,
             70,  71,  78,  79,  87,  88,  96,  97, 101, 105, 106, 110, 115,
            118, 119, 123, 124, 125, 127, 128, 131, 132, 133, 134, 136, 137,
            140, 141, 145, 146, 149, 150, 154, 155, 158, 159, 161, 162, 163,
            164, 166, 167, 170, 171, 172, 173, 174, 175, 176, 177, 179],
           dtype='int64')

In [0]:
postal_data.drop(unassigned_borough, inplace=True)

In [50]:
postal_data

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
...,...,...,...
160,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Processing CentrE
169,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...


2. More than one neighborhood can exist in one postal code area. Such rows will be combined into one row with the neighborhoods separated with a comma

In [42]:
print(postal_data.head())
postal_data_clean = postal_data.copy(deep=True)
for i in range(103):
  postal_data_clean["Neighborhood"].iloc[i] = postal_data_clean["Neighborhood"].iloc[i].replace("/",",")
postal_data_clean

  Postal code           Borough                                  Neighborhood
2         M3A        North York                                     Parkwoods
3         M4A        North York                              Victoria Village
4         M5A  Downtown Toronto                    Regent Park / Harbourfront
5         M6A        North York             Lawrence Manor / Lawrence Heights
6         M7A  Downtown Toronto  Queen's Park / Ontario Provincial Government


,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Processing CentrE
169,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,..."


3. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [43]:
postal_data_clean[postal_data_clean["Neighborhood"] == "Not Assigned"]

,Postal code,Borough,Neighborhood


As we can see there are no such Neighborhoods

In [52]:
df_new = postal_data_clean.reset_index(drop = True)
df_new.head(15)

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern , Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill , Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Finally, shape of the df

In [53]:
df_new.shape

(103, 3)

### 2. Use the Geocoder package or the csv file to create dataframe with longitude and latitude values 


We need to get the latitude and the longitude coordinates of each neighborhood. 

Installing Geocoder library

In [55]:
!pip install geocoder

     |████████████████████████████████| 102kB 2.3MB/s 


Since the geocoder library is unreliable we will import the latitude and longitudes from csv file. 

Changing column name so that it is same for both dataframes

In [65]:
df_ll = pd.read_csv("https://cocl.us/Geospatial_data")
df_ll.rename(columns = {"Postal Code":"Postal code"}, inplace = True)
df_ll.head()

,Postal code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging the two dataframes

In [68]:
df_complete = pd.merge(df_new, df_ll, on = "Postal code", how = "outer")
df_complete.head(15)

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [69]:
df_complete.shape

(103, 5)

### 3. Explore and cluster the neighborhoods in Toronto

Clustering boroughs that contain the word Toronto.

Importing Libraries

In [0]:
from sklearn.cluster import KMeans
import folium 

Visualising all boroughs on map



In [80]:
map_toronto = folium.Map([43.7238, -79.3832], zoom_start = 12)

for lat, lng, borough, neighborhood in zip(df_complete["Latitude"], df_complete["Longitude"], df_complete["Borough"], df_complete["Neighborhood"]):
  label = '{}, {}'.format(neighborhood, borough)
  label = folium.Popup(label, parse_html=True)
  folium.CircleMarker(
      [lat, lng],
      radius = 5,
      popup = label,
      color='green',
      fill=True,
      fill_color='#3146cc',
      fill_opacity=0.7,
      parse_html=False).add_to(map_toronto)

      
map_toronto

However, for illustration purposes, let's simplify the above map and segment and cluster only the boroughs that contain the word Toronto.

In [82]:
df_toronto = df_complete[df_complete["Borough"].str.find("Toronto") >= 0].reset_index(drop = True)
df_toronto.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


Now lets Visualize all the boroughs that contain the word Toronto

In [83]:
map_toronto_b = folium.Map([43.7238, -79.3832], zoom_start = 12)

for lat, lng, borough, neighborhood in zip(df_toronto["Latitude"], df_toronto["Longitude"], df_toronto["Borough"], df_toronto["Neighborhood"]):
  label = '{}, {}'.format(neighborhood, borough)
  label = folium.Popup(label, parse_html=True)
  folium.CircleMarker(
      [lat, lng],
      radius = 5,
      popup = label,
      color='blue',
      fill=True,
      fill_color='#3146cc',
      fill_opacity=0.7,
      parse_html=False).add_to(map_toronto_b)

      
map_toronto_b

All the Boroughs

In [84]:
df_toronto["Borough"].unique()

array(['Downtown Toronto', 'East Toronto', 'West Toronto',
       'Central Toronto'], dtype=object)

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.


Define Foursquare Credentials and Version


In [0]:
CLIENT_ID = 'CZARKFFI14PYXT3RTONKB0N4BVDATC4OQ5PIDLUPA4TWOUZ4' 
CLIENT_SECRET = 'TT2BQK2Z3T55EVQFGVZ3YN1CIPVUVUQLPNW5MIZEF2QPY2AB' 
VERSION = '20180605'

Function to Explore Neighborhoods in Toronto

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=300):
    
    LIMIT = 50
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [90]:
toronto_venues = getNearbyVenues(names = df_toronto["Neighborhood"], 
                                 latitudes = df_toronto["Latitude"],
                                 longitudes = df_toronto["Longitude"],
                                )
print(toronto_venues.shape)
print(toronto_venues.head())

Regent Park , Harbourfront
Queen's Park , Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond , Adelaide , King
Dufferin , Dovercourt Village
Harbourfront East , Union Station , Toronto Islands
Little Portugal , Trinity
The Danforth West , Riverdale
Toronto Dominion Centre , Design Exchange
Brockton , Parkdale Village , Exhibition Place
India Bazaar , The Beaches West
Commerce Court , Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park , The Junction South
North Toronto West
The Annex , North Midtown , Yorkville
Parkdale , Roncesvalles
Davisville
University of Toronto , Harbord
Runnymede , Swansea
Moore Park , Summerhill East
Kensington Market , Chinatown , Grange Park
Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport
Roseda

Venues for each neighborhood 

In [93]:
toronto_venues[["Neighborhood","Venue Category"]].groupby('Neighborhood').count()

,Venue Category
Neighborhood,
Berczy Park,7
"Brockton , Parkdale Village , Exhibition Place",13
Business reply mail Processing CentrE,7
"CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport",5
Central Bay Street,32
Christie,6
Church and Wellesley,50
"Commerce Court , Victoria Hotel",50
Davisville,21


In [95]:
len(toronto_venues["Venue Category"].unique())

180

#### Analysing each Neighborhood

In [96]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arepa Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bakery,Bank,Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,...,Plaza,Poke Place,Pool,Pub,Ramen Restaurant,Record Shop,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category¶


In [98]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arepa Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bakery,Bank,Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,...,Plaza,Poke Place,Pool,Pub,Ramen Restaurant,Record Shop,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,Berczy Park,0.000000,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.142857,...,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.142857,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00
1,"Brockton , Parkdale Village , Exhibition Place",0.000000,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.153846,0.000000,0.00,0.00,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.076923,0.00,0.00,0.153846,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.076923,0.000000,0.00,0.00
2,Business reply mail Processing CentrE,0.000000,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.142857,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00
3,"CN Tower , King and Spadina , Railway Lands , ...",0.000000,0.00,0.2,0.2,0.2,0.2,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00
4,Central Bay Street,0.000000,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.0000,0.000000,0.031250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.031250,0.000000,0.000000,0.000000,0.000000,0.031250,0.00,0.00,0.000000,0.000000,0.062500,0.000000,0.00,0.00,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.031250,0.00,0.00,0.062500,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.031250,0.00,0

Neighborhood  with the top 5 most common venues

In [99]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0    Concert Hall  0.14
1  Clothing Store  0.14
2        Beer Bar  0.14
3      Restaurant  0.14
4    Liquor Store  0.14


----Brockton , Parkdale Village , Exhibition Place----
                 venue  freq
0                 Café  0.15
1       Sandwich Place  0.15
2                  Gym  0.08
3  Japanese Restaurant  0.08
4            Pet Store  0.08


----Business reply mail Processing CentrE----
                  venue  freq
0                  Park  0.14
1                Garden  0.14
2               Brewery  0.14
3  Fast Food Restaurant  0.14
4        Farmers Market  0.14


----CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport----
                venue  freq
0  Airport Food Court   0.2
1        Airport Gate   0.2
2      Airport Lounge   0.2
3    Airport Terminal   0.2
4         Coffee Shop   0.2


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.41
1

The top 10 venues for each neighborhood.

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [102]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Concert Hall,Grocery Store,Breakfast Spot,Liquor Store,Clothing Store,Beer Bar,Restaurant,Wings Joint,Distribution Center,Electronics Store
1,"Brockton , Parkdale Village , Exhibition Place",Sandwich Place,Café,Coffee Shop,Park,Japanese Restaurant,Restaurant,Pet Store,Breakfast Spot,Gym,Grocery Store
2,Business reply mail Processing CentrE,Park,Garden,Brewery,Light Rail Station,Fast Food Restaurant,Auto Workshop,Farmers Market,Gas Station,Department Store,Dumpling Restaurant
3,"CN Tower , King and Spadina , Railway Lands , ...",Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,Coffee Shop,Wings Joint,Diner,Falafel Restaurant,Ethiopian Restaurant,Electronics Store
4,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Deli / Bodega,Bubble Tea Shop,Bookstore,Spa,Restaurant,Bank


### Cluster Neighborhoods

In [104]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:20]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 2],
      dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.



In [108]:
# neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# # merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,0.0,Furniture / Home Store,Breakfast Spot,Distribution Center,Food Truck,Gastropub,Bakery,Gym,Gym / Fitness Center,Mediterranean Restaurant,History Museum
1,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494,0.0,Coffee Shop,Italian Restaurant,Thai Restaurant,Sandwich Place,Bike Rental / Bike Share,Bubble Tea Shop,Sushi Restaurant,Café,Sculpture Garden,Park
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0.0,Coffee Shop,Middle Eastern Restaurant,Café,Clothing Store,Restaurant,Tea Room,Greek Restaurant,Diner,Electronics Store,Hookah Bar
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0.0,Gastropub,Coffee Shop,Japanese Restaurant,Food Truck,BBQ Joint,Restaurant,Hotel,Italian Restaurant,Middle Eastern Restaurant,Café
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0.0,Park,Trail,Other Great Outdoors,Comic Shop,Dessert Shop,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Dog Run


In [0]:
toronto_merged["Cluster Labels"].isnull().values.any()
toronto_merged.dropna(inplace=True)

Lets see it on a map

In [119]:
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map([43.7238, -79.3832], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster + 0.5)-1],
        fill=True,
        fill_color=rainbow[int(cluster + 0.5)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

In [122]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0.0,Furniture / Home Store,Breakfast Spot,Distribution Center,Food Truck,Gastropub,Bakery,Gym,Gym / Fitness Center,Mediterranean Restaurant,History Museum
1,Downtown Toronto,0.0,Coffee Shop,Italian Restaurant,Thai Restaurant,Sandwich Place,Bike Rental / Bike Share,Bubble Tea Shop,Sushi Restaurant,Café,Sculpture Garden,Park
2,Downtown Toronto,0.0,Coffee Shop,Middle Eastern Restaurant,Café,Clothing Store,Restaurant,Tea Room,Greek Restaurant,Diner,Electronics Store,Hookah Bar
3,Downtown Toronto,0.0,Gastropub,Coffee Shop,Japanese Restaurant,Food Truck,BBQ Joint,Restaurant,Hotel,Italian Restaurant,Middle Eastern Restaurant,Café
4,East Toronto,0.0,Park,Trail,Other Great Outdoors,Comic Shop,Dessert Shop,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Dog Run
5,Downtown Toronto,0.0,Concert Hall,Grocery Store,Breakfast Spot,Liquor Store,Clothing Store,Beer Bar,Restaurant,Wings Joint,Distribution Center,Electronics Store
6,Downtown Toronto,0.0,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Deli / Bodega,Bubble Tea Shop,Bookstore,Spa,Restaurant,Bank
7,Downtown Toronto,0.0,Grocery Store,Café,Gym / Fitness Center,Coffee Shop,Candy Store,Wings Joint,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop
8,Downtown Toronto,0.0,Asian Restaurant,Steakhouse,Japanese Restaurant,Seafood Restaurant,Restaurant,Pizza Place,Sushi Restaurant,Coffee Shop,Café,Salad Place
9,West Toronto,0.0,Liquor Store,Gym / Fitness Center,Bank,Bakery,Middle Eastern Restaurant,Pharmacy,Pool,Bar,Grocery Store,Gas Station


In [123]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,1.0,Health & Beauty Service,Wings Joint,Dessert Shop,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Dog Run,Distribution Center


In [124]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,2.0,Sushi Restaurant,Wings Joint,Diner,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Dog Run,Distribution Center


In [125]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Downtown Toronto,3.0,Park,Building,Farmers Market,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store


In [126]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Central Toronto,4.0,Park,Intersection,Dessert Shop,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store
